In [27]:
from pyrosetta import *
from pyrosetta.rosetta import *
init()

Found rosetta database at: /usr/local/lib/python2.7/dist-packages/pyrosetta-2018.11+release.7c7790f-py2.7-linux-x86_64.egg/pyrosetta/database; using it....
PyRosetta-4 2017 [Rosetta PyRosetta4.Release.python27.ubuntu 2018.11+release.7c7790f7c7790fbb898f8210b54513b9f9c8a927c3e8d7f 2018-03-14T13:19:04] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.

core.init: Checking for fconfig files in pwd and ./rosetta/flags

core.init: Rosetta version: PyRosetta4.Release.python27.ubuntu r173 2018.11+release.7c7790f 7c7790fbb898f8210b54513b9f9c8a927c3e8d7f http://www.pyrosetta.org 2018-03-14T13:19:04
core.init: command: PyRosetta -ex1 -ex2aro -database /usr/local/lib/python2.7/dist-packages/pyrosetta-2018.11+release.7c7790f-py2.7-linux-x86_64.egg/pyrosetta/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=-128573177 seed_offset=0 real_seed=-128573177
core.init.random: RandomGenerator

In [28]:
import math
import random
import numpy as np
import scipy
import matplotlib
import matplotlib.pyplot as plt
from pyrosetta.rosetta.protocols.moves import *
from pyrosetta.rosetta.core.id import *

In [29]:
def within_360(ang):
    
    return ang - 360.0*int(ang/360.0)

In [30]:
def hessian(x, h):
        
    x_grad = np.gradient(x, h)
    hessian = np.empty((x.ndim, x.ndim) + x.shape, dtype = x.dtype)
    
    for k, grad_k in enumerate(x_grad):
        tmp_grad = np.gradient(grad_k, h)
        for l, grad_kl in enumerate(tmp_grad):
            hessian[k, l, :, :] = grad_kl
            
    return hessian

In [70]:
def initialize_pose(seq1, seq2):
    
    n_residues = len(seq1) + len(seq2)
    
    dof_dict = {}
    
    pose = pyrosetta.rosetta.protocols.recces.pose_setup_turner(seq1, seq2)
    movemap = MoveMap()
    
    for ii in range(n_residues-1):
    
        dof_dict.update({6*ii:TorsionID(ii+2, BB, 1), 6*ii+1:TorsionID(ii+2, BB, 2), 6*ii+2:TorsionID(ii+2, BB, 3),
                 6*ii+3:TorsionID(ii+1, BB, 5), 6*ii+4:TorsionID(ii+1, BB, 6)})
        
        movemap.set(TorsionID(ii+2, BB, 1), True)
        movemap.set(TorsionID(ii+2, BB, 2), True)
        movemap.set(TorsionID(ii+2, BB, 3), True)
        movemap.set(TorsionID(ii+1, BB, 5), True)
        movemap.set(TorsionID(ii+1, BB, 6), True)
        
    for ii in range(n_residues):
        
        dof_dict.update({6*ii+5:TorsionID(ii+1, CHI, 1)})
        movemap.set(TorsionID(ii+1, CHI, 1), True)
    
    return pose, movemap, dof_dict

In [74]:
def find_minimum(pose, sf, movemap):
    
    
    minimum = Pose()
    minimum.assign(pose)
    #observer = AddPyMOLObserver(minimum, True)
    sf(minimum)
    n_residues = pose.total_residue()
    
    movemap = MoveMap()
    for ii in range(n_residues-1):
        
        minimum.set_alpha(  ii+2, random.uniform(0, 360))
        minimum.set_beta(   ii+2, random.uniform(0, 360))
        minimum.set_gamma(  ii+2, random.uniform(0, 360))
        minimum.set_epsilon(ii+1, random.uniform(0, 360))
        minimum.set_zeta(   ii+1, random.uniform(0, 360))
             
    for ii in range(n_residues):
        
        pose.set_chi(ii+1, random.uniform(0, 360))
    '''
    minmap = rosetta.core.optimization.MinimizerMap()
    minmap.setup(pose, movemap)
    sf.setup_for_minimizing(pose, minmap)
    multifunc = rosetta.core.optimization.AtomTreeMultifunc(pose, minmap, sf)
    num_dofs = minmap.nangles()
    print num_dofs
    '''
    minmover = rosetta.protocols.minimization_packing.MinMover(movemap, sf, 'dfpmin', 1e-5, True)
    
    minmover.apply(minimum)
    
    return minimum   

In [78]:
pose = initialize_pose('aa', '')
scorefxn = core.scoring.ScoreFunctionFactory.create_score_function('rna/denovo/rna_hires')

In [77]:
pmm = PyMOLMover()
pmm.apply(pose)

In [24]:
import itertools

trial = Pose()
trial.assign(pose)

h = 0.5
h_rad = h*np.pi/180

dofs = []
dofs.append(pose.alpha(2))
dofs.append(pose.beta(2))
dofs.append(pose.gamma(2))
dofs.append(pose.epsilon(1))
dofs.append(pose.zeta(1))
dofs.append(pose.chi(1))
dofs.append(pose.chi(2))

tor_ranges = np.zeros((int(4/h + 1), len(dofs)))
print tor_ranges.shape
print np.arange(-2, 2+h, h) 

for ii, dof in enumerate(dofs):
    
    tor_ranges[:, ii] = dof + np.arange(-2, 2+h, h)
    
for pair in list(itertools.combinations(range(len(dofs)), 2)):
    
    x = tor_ranges[pair[0],:]
    y = tor_ranges[pair[1],:]
    
    for ii, xx in x:
        
        pose.set
        for jj, yy in y:
            
            pose.
            energy[ii, jj] = scorefxn
    
    
               
                      
    
    

SyntaxError: invalid syntax (<ipython-input-24-4be99ef78eed>, line 23)

In [23]:
print tor_ranges

[[  -4.39436853  178.73155028  321.92391229   53.32846747  244.47231346
    74.79481502   92.35013594]
 [  -3.89436853  179.23155028  322.42391229   53.82846747  244.97231346
    75.29481502   92.85013594]
 [  -3.39436853  179.73155028  322.92391229   54.32846747  245.47231346
    75.79481502   93.35013594]
 [  -2.89436853  180.23155028  323.42391229   54.82846747  245.97231346
    76.29481502   93.85013594]
 [  -2.39436853  180.73155028  323.92391229   55.32846747  246.47231346
    76.79481502   94.35013594]
 [  -1.89436853  181.23155028  324.42391229   55.82846747  246.97231346
    77.29481502   94.85013594]
 [  -1.39436853  181.73155028  324.92391229   56.32846747  247.47231346
    77.79481502   95.35013594]
 [  -0.89436853  182.23155028  325.42391229   56.82846747  247.97231346
    78.29481502   95.85013594]
 [  -0.39436853  182.73155028  325.92391229   57.32846747  248.47231346
    78.79481502   96.35013594]]
